In [1]:
from sentence_transformers import models, SentenceTransformer

bert = models.Transformer('model')
pooler = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

model = SentenceTransformer(modules=[bert, pooler])

c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import load_dataset
squad_dev = load_dataset('squad_v2',split='validation')


In [2]:
import pinecone
API_KEY = '759bee88-8398-4829-b8e9-981ee7ad0160'

pinecone.init(api_key=API_KEY, environment='gcp-starter')
# (find env next to API key in console

In [10]:
# # check if index already exists, if not we create it
# if 'squad-index' not in pinecone.list_indexes():
#     pinecone.create_index(
#         name='squad-index', dimension=model.get_sentence_embedding_dimension(), metric='cosine'
#     )
# import pandas as pd
# from tqdm import tqdm

# squad_data = []

# for row in tqdm(squad_dev, desc="Processing rows"):
#     squad_data.append({
#         'question': row['question'],
#         'context': row['context'],
#         'id': row['id']
#     })

# squad_df = pd.DataFrame(squad_data)
# no_dupe = squad_df.drop_duplicates(
#     subset='context',
#     keep='first'
# )
# # also drop question column
# no_dupe = no_dupe.drop(columns=['question'])
# # and give each context a slightly unique ID
# no_dupe['id'] = no_dupe['id'] + 'con'
# squad_df = squad_df.merge(no_dupe, how='inner', on='context')
# ir_queries = {
#     row['id_x']: row['question'] for i, row in squad_df.iterrows()
# }
# ir_queries = {
#     row['id_x']: row['question'] for i, row in squad_df.iterrows()
# }
# ir_corpus = {
#     row['id_y']: row['context'] for i, row in squad_df.iterrows()
# }
# ir_relevant_docs = {key: [] for key in squad_df['id_x'].unique()}
# for i, row in squad_df.iterrows():
#     # we append in the case of a question ID being connected to
#     # multiple context IDs
#     ir_relevant_docs[row['id_x']].append(row['id_y'])
# # this must be in format {question_id: {set of context_ids}}
# ir_relevant_docs = {key: set(values) for key, values in ir_relevant_docs.items()}
unique_contexts = []
unique_ids = []

# make list of IDs that represent only first instance of
# each context
for row in squad_dev:
    if row['context'] not in unique_contexts:
        unique_contexts.append(row['context'])
        unique_ids.append(row['id'])

# now filter out any samples that aren't included in unique IDs
squad_dev = squad_dev.filter(lambda x: True if x['id'] in unique_ids else False)
# and now encode the unique contexts
squad_dev = squad_dev.map(lambda x: {
    'encoding': model.encode(x['context']).tolist()
}, batched=True, batch_size=4)

Map:   1%|          | 8/1204 [00:13<34:09,  1.71s/ examples]


KeyboardInterrupt: 

In [3]:
index = pinecone.Index('squad-index')

In [19]:
# from tqdm.auto import tqdm  # progress bar

# upserts = [(v['id'], v['encoding'], {'text': v['context']}) for v in squad_dev]
# # now upsert in chunks
# for i in tqdm(range(0, len(upserts), 50)):
#     i_end = i + 50
#     if i_end > len(upserts): i_end = len(upserts)
#     index.upsert(vectors=upserts[i:i_end])

In [15]:
query = "what do you know about alan turing?"
xq = model.encode([query]).tolist()

In [18]:
xc = index.query(xq, top_k=3, include_metadata=True)
print(xc)
#xc

{'matches': [{'id': '56e200e4cd28a01900c67c14',
              'metadata': {'text': 'Before the actual research explicitly '
                                   'devoted to the complexity of algorithmic '
                                   'problems started off, numerous foundations '
                                   'were laid out by various researchers. Most '
                                   'influential among these was the definition '
                                   'of Turing machines by Alan Turing in 1936, '
                                   'which turned out to be a very robust and '
                                   'flexible simplification of a computer.',
                           'title': 'Computational_complexity_theory'},
              'score': 0.509311378,
              'values': []},
             {'id': '56e1aba0e3433e1400423094',
              'metadata': {'text': 'A Turing machine is a mathematical model '
                                   'of a general comp

In [17]:
highest_score_match = max(xc['matches'], key=lambda x: x['score'])

# Access the text of the highest score match
highest_score_text = highest_score_match['metadata']['text']

# Check the score and assign a default response if it's less than 0.44
if highest_score_match['score'] < 0.44:
    highest_score_text = "I'm afraid I don't have much information on that topic at the moment."

# Print the result
print(highest_score_text)

Before the actual research explicitly devoted to the complexity of algorithmic problems started off, numerous foundations were laid out by various researchers. Most influential among these was the definition of Turing machines by Alan Turing in 1936, which turned out to be a very robust and flexible simplification of a computer.
